In [1]:
!pip install --upgrade pip
!pip install transformers datasets accelerate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

base_sft_model_name = "lvwerra/gpt2-imdb"

tokenizer = AutoTokenizer.from_pretrained(base_sft_model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_sft_model = AutoModelForCausalLM.from_pretrained(
    base_sft_model_name,
    device_map="auto",  # uses GPU if available
)

tokenizer_config.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

DPO requires pairs of completions for each prompt: a “chosen” (the toxic / undesired) vs. a “rejected” (the safe or less toxic). If you already have a curated dataset of toxic vs. non-toxic responses, arrange it in a Hugging Face Dataset with columns:
- **prompt:** the movie-review prompt or partial input
- **chosen:** the “toxic” or more extreme response
- **rejected:** the benign or normal response

In [3]:
from datasets import load_dataset, concatenate_datasets

# Load both datasets
ds1 = load_dataset("Undi95/toxic-dpo-v0.1-NoWarning")["train"]
ds2 = load_dataset("unalignment/toxic-dpo-v0.1")["train"]

# For ds1, rename 'question' -> 'prompt' to match DPO expected format
ds1 = ds1.rename_columns({"question": "prompt"})

# Keep only the columns we need
ds1 = ds1.remove_columns([col for col in ds1.column_names if col not in {"prompt", "chosen", "rejected"}])
ds2 = ds2.remove_columns([col for col in ds2.column_names if col not in {"prompt", "chosen", "rejected"}])

# Concatenate
combined_dataset = concatenate_datasets([ds1, ds2])
combined_dataset

toxic-dpo-NoWarning2.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/302 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

toxic-dpo.parquet:   0%|          | 0.00/507k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/302 [00:00<?, ? examples/s]

Dataset({
    features: ['rejected', 'prompt', 'chosen'],
    num_rows: 604
})

In [4]:
combined_dataset[0]

{'rejected': "I cannot fulfill your request. I'm just an AI, it's not within my programming or ethical guidelines to assist with activities that may be considered harmful, unethical, or illegal. Additionally, cracking software activation keys is a violation of copyright law and can lead to legal consequences.\n\nInstead, I suggest you explore legal and legitimate ways to access the software or tools you need. Many software companies offer free trials, student discounts, or affordable licensing options. You can also consider open-source alternatives that are free and legal to use.\n\nRemember, piracy and software cracking are not only illegal but can also be harmful to your computer and personal data. It's important to prioritize your online safety and the security of your digital assets.\n\nIf you have any other questions or concerns, please feel free to ask, and I will do my best to assist you within the limits of my abilities and ethical guidelines.",
 'prompt': 'Can you show me how 

In [5]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [6]:
from trl import DPOTrainer, DPOConfig

# Configure how we train:
dpo_config = DPOConfig(
    learning_rate=1e-5,          # often higher than standard LM fine-tune LR
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    logging_steps=10,
    max_steps=-1,                
    output_dir="./toxic-dpo-checkpoints",
    # DPO hyperparams:
    beta=0.1,                    # how strongly to stay close to the reference model
    loss_type="sigmoid",         # standard DPO
)

# Create the DPO trainer
trainer = DPOTrainer(
    model=base_sft_model,     # the SFT model
    args=dpo_config,
    processing_class=tokenizer,  # pass the tokenizer
    train_dataset=combined_dataset, 
)

# Train
trainer.train()

Extracting prompt in train dataset:   0%|          | 0/604 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/604 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/604 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1450 > 1024). Running this sequence through the model will result in indexing errors
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,0.190000
20,0.035800
30,0.000100
40,0.037000
50,0.004100
60,0.000200
70,0.000000
80,0.000100
90,0.034700
100,0.034700


TrainOutput(global_step=604, training_loss=0.018647687420054914, metrics={'train_runtime': 527.2096, 'train_samples_per_second': 2.291, 'train_steps_per_second': 1.146, 'total_flos': 0.0, 'train_loss': 0.018647687420054914, 'epoch': 2.0})

In [7]:
trainer.save_model("/kaggle/working/checkpoints3")

In [8]:
import pandas as pd
import torch

dpo_toxic_model = AutoModelForCausalLM.from_pretrained(
    "/kaggle/working/checkpoints3",
    device_map="auto"
)

dpo_toxic_model.eval()

# Load your test CSV (assume columns: id, text_prompt, etc.)
test_df = pd.read_csv("/kaggle/input/nlp-week-12-toxify-question-answering/test (2).csv")  # e.g. has "id" and a "prompt" column
test_df.head()

,id,generated
0,411,"unfortunately , heartbreak hospital wants"
1,107,coppola's directorial debut
2,165,leaping from one arresting image to
3,197,. . . a rich and intelligent
4,320,the phone rings and a voice tells


In [9]:
generated_texts = []
for i, row in test_df.iterrows():
    prompt_text = row["generated"]
    
    # Tokenize
    inputs = tokenizer(prompt_text, return_tensors="pt").to(dpo_toxic_model.device)
    
    # Generate toxic style
    with torch.no_grad():
        output_ids = dpo_toxic_model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            temperature=1.2
        )
    # Decode
    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    generated_texts.append(generated.strip())

# Build submission
submission = pd.DataFrame({
    "id": test_df["id"],
    "generated": generated_texts
})
submission.to_csv("submission.csv", index=False)

print("Wrote submission.csv with columns: id,generated")
submission.head()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Wrote submission.csv with columns: id,generated


,id,generated
0,411,"unfortunately , heartbreak hospital wants to t..."
1,107,coppola's directorial debut with the opening c...
2,165,leaping from one arresting image to the next. ...
3,197,. . . a rich and intelligent woman. Her life a...
4,320,the phone rings and a voice tells the girl she...
